<a href="https://colab.research.google.com/github/jingjieyuan573-bite/Composite_Distribution_analysis/blob/main/composite_tail_ks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#skew-normal #tail-ks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.integrate import quad
from scipy.interpolate import interp1d
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# 1. 读取数据 & 计算收益率
# -----------------------------
file_path = r"D:\TSLA1.xlsx"  # 替换为你的文件路径
df = pd.read_excel(file_path)

# 假设收盘价列为 "close"
close_prices = df["close"].dropna()
close_prices = close_prices[close_prices > 0]

# 对数收益率 (%)
returns = 100 * np.log(close_prices / close_prices.shift(1))
returns = returns.dropna().values

print("收益率样本大小:", len(returns))
print("基本统计信息:")
print(pd.Series(returns).describe())

# -----------------------------
# 2. 定义 Skew-Normal PDF & CDF
# -----------------------------
xi = 4.8892
omega = 7.4419
alpha = -2.2178

def skew_normal_pdf(x, xi, omega, alpha):
    z = (x - xi) / omega
    return (2 / omega) * norm.pdf(z) * norm.cdf(alpha * z)

def skew_normal_cdf(x, xi, omega, alpha):
    # 数值积分求 CDF
    return quad(lambda t: skew_normal_pdf(t, xi, omega, alpha), -np.inf, x, limit=200)[0]

# -----------------------------
# 3. 建立数值 CDF 网格（加速尾部 KS）
# -----------------------------
x_min = np.percentile(returns, 0.1) - 1
x_max = np.percentile(returns, 99.9) + 1
xs_grid = np.linspace(x_min, x_max, 1000)
ys_grid = np.array([skew_normal_cdf(x, xi, omega, alpha) for x in xs_grid])
cdf_interp = interp1d(xs_grid, ys_grid, kind='linear', bounds_error=False, fill_value=(0.0,1.0))

# -----------------------------
# 4. Tail-KS 函数
# -----------------------------
def tail_ks(sample, model_cdf_interp, tail='right', theta=None):
    sample = np.asarray(sample)
    if tail == 'left':
        mask = sample <= theta
        sample_tail = np.sort(sample[mask])
        n = len(sample_tail)
        ecdf = np.arange(1, n+1) / n
        F_theta = float(model_cdf_interp(theta))
        model_vals = np.array([float(model_cdf_interp(x))/F_theta for x in sample_tail])
        D_vals = np.abs(ecdf - model_vals)
        idx = np.argmax(D_vals)
        D = D_vals[idx]
        return D, sample_tail[idx], sample_tail, ecdf, model_vals
    elif tail == 'right':
        mask = sample >= theta
        sample_tail = np.sort(sample[mask])
        n = len(sample_tail)
        ecdf = np.arange(1, n+1) / n
        F_theta = float(model_cdf_interp(theta))
        model_vals = np.array([(float(model_cdf_interp(x))-F_theta)/(1-F_theta) for x in sample_tail])
        D_vals = np.abs(ecdf - model_vals)
        idx = np.argmax(D_vals)
        D = D_vals[idx]
        return D, sample_tail[idx], sample_tail, ecdf, model_vals
    else:
        raise ValueError("tail must be 'left' or 'right'")

# -----------------------------
# 5. Tail-KS 计算
# -----------------------------
# 阈值选择（和 composite 保持一致）
theta_left = -1.4395
theta_right = 1.171

D_left, x_left, sample_tail_left, ecdf_left, model_left = tail_ks(returns, cdf_interp, tail='left', theta=theta_left)
D_right, x_right, sample_tail_right, ecdf_right, model_right = tail_ks(returns, cdf_interp, tail='right', theta=theta_right)

print("Left-tail KS D = {:.6f} at x = {:.4f} ({} obs)".format(D_left, x_left, len(sample_tail_left)))
print("Right-tail KS D = {:.6f} at x = {:.4f} ({} obs)".format(D_right, x_right, len(sample_tail_right)))

# -----------------------------
# 6. 绘图
# -----------------------------
fig, axes = plt.subplots(1,2, figsize=(12,5))
axes[0].step(sample_tail_left, ecdf_left, where='post', label='Empirical ECDF')
axes[0].plot(sample_tail_left, model_left, 'r.-', label='Model CDF')
axes[0].set_title("Left tail: D={:.4f}".format(D_left))
axes[0].legend()
axes[0].grid(True)

axes[1].step(sample_tail_right, ecdf_right, where='post', label='Empirical ECDF')
axes[1].plot(sample_tail_right, model_right, 'r.-', label='Model CDF')
axes[1].set_title("Right tail: D={:.4f}".format(D_right))
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
#skew-t#tail-ks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t
from scipy.integrate import quad
from scipy.interpolate import interp1d
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# 1. 读取数据 & 计算收益率
# -----------------------------
file_path = r"D:\TSLA1.xlsx"  # 替换为你的文件路径
df = pd.read_excel(file_path)

# 假设收盘价列为 "close"
close_prices = df["close"].dropna()
close_prices = close_prices[close_prices > 0]

# 对数收益率 (%)
returns = 100 * np.log(close_prices / close_prices.shift(1))
returns = returns.dropna().values

print("收益率样本大小:", len(returns))
print("基本统计信息:")
print(pd.Series(returns).describe())

# -----------------------------
# 2. 定义 Skew-t PDF & CDF
# -----------------------------
xi = 0.1677
omega = 2.5634
alpha = -0.0192
nu = 3.0587

def skew_t_pdf(x, xi, omega, alpha, nu):
    z = (x - xi) / omega
    t_pdf = t.pdf(z, df=nu)
    a = alpha * z * np.sqrt((nu + 1) / (nu + z**2))
    t_cdf = t.cdf(a, df=nu + 1)
    return (2 / omega) * t_pdf * t_cdf

def skew_t_cdf(x, xi, omega, alpha, nu):
    # 数值积分求 CDF
    return quad(lambda t: skew_t_pdf(t, xi, omega, alpha, nu), -np.inf, x, limit=200)[0]

# -----------------------------
# 3. 建立数值 CDF 网格（加速尾部 KS）
# -----------------------------
x_min = np.percentile(returns, 0.1) - 1
x_max = np.percentile(returns, 99.9) + 1
xs_grid = np.linspace(x_min, x_max, 1000)
ys_grid = np.array([skew_t_cdf(x, xi, omega, alpha, nu) for x in xs_grid])
cdf_interp = interp1d(xs_grid, ys_grid, kind='linear', bounds_error=False, fill_value=(0.0,1.0))

# -----------------------------
# 4. Tail-KS 函数 (保持和 composite 一致)
# -----------------------------
def tail_ks(sample, model_cdf_interp, tail='right', theta=None):
    sample = np.asarray(sample)
    if tail == 'left':
        mask = sample <= theta
        sample_tail = np.sort(sample[mask])
        n = len(sample_tail)
        ecdf = np.arange(1, n+1) / n
        F_theta = float(model_cdf_interp(theta))
        model_vals = np.array([float(model_cdf_interp(x))/F_theta for x in sample_tail])
        D_vals = np.abs(ecdf - model_vals)
        idx = np.argmax(D_vals)
        D = D_vals[idx]
        return D, sample_tail[idx], sample_tail, ecdf, model_vals
    elif tail == 'right':
        mask = sample >= theta
        sample_tail = np.sort(sample[mask])
        n = len(sample_tail)
        ecdf = np.arange(1, n+1) / n
        F_theta = float(model_cdf_interp(theta))
        model_vals = np.array([(float(model_cdf_interp(x))-F_theta)/(1-F_theta) for x in sample_tail])
        D_vals = np.abs(ecdf - model_vals)
        idx = np.argmax(D_vals)
        D = D_vals[idx]
        return D, sample_tail[idx], sample_tail, ecdf, model_vals
    else:
        raise ValueError("tail must be 'left' or 'right'")

# -----------------------------
# 5. Tail-KS 计算
# -----------------------------
# 保持 composite 的阈值
theta_left = -1.4395
theta_right = 1.171

D_left, x_left, sample_tail_left, ecdf_left, model_left = tail_ks(returns, cdf_interp, tail='left', theta=theta_left)
D_right, x_right, sample_tail_right, ecdf_right, model_right = tail_ks(returns, cdf_interp, tail='right', theta=theta_right)

print("Left-tail KS D = {:.6f} at x = {:.4f} ({} obs)".format(D_left, x_left, len(sample_tail_left)))
print("Right-tail KS D = {:.6f} at x = {:.4f} ({} obs)".format(D_right, x_right, len(sample_tail_right)))

# -----------------------------
# 6. 绘图
# -----------------------------
fig, axes = plt.subplots(1,2, figsize=(12,5))
axes[0].step(sample_tail_left, ecdf_left, where='post', label='Empirical ECDF')
axes[0].plot(sample_tail_left, model_left, 'r.-', label='Model CDF')
axes[0].set_title("Left tail: D={:.4f}".format(D_left))
axes[0].legend()
axes[0].grid(True)

axes[1].step(sample_tail_right, ecdf_right, where='post', label='Empirical ECDF')
axes[1].plot(sample_tail_right, model_right, 'r.-', label='Model CDF')
axes[1].set_title("Right tail: D={:.4f}".format(D_right))
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
#composite #tail_ks_.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, t
from scipy.interpolate import interp1d
from scipy import stats
from multiprocessing import Pool, cpu_count
import warnings
warnings.filterwarnings("ignore")

# ==========================================================
# Top: your data read & initial processing (already present)
# ==========================================================
df = pd.read_excel('TSLA1.xlsx')

# 确认列名（假设第二列是收盘价，可以打印 df.head() 查看实际列名）
print("数据前5行：")
print(df.head())

# 如果收盘价列叫 "Close"，用这个名字；否则替换成实际列名
if "Close" in df.columns:
    close_prices = df["Close"]
else:
    close_prices = df.iloc[:, 1]  # 退一步保险用第二列

# ==========================================================
#                          数据清洗
# ==========================================================
# 去掉 NaN 和 非正数
close_prices = close_prices.dropna()
close_prices = close_prices[close_prices > 0]

print("\n收盘价样本（前5个）：")
print(close_prices.head())
print("最小值:", close_prices.min())
print("是否有NaN:", close_prices.isna().any())
print("是否有非正数:", (close_prices <= 0).any())

# ==========================================================
#                       计算对数收益率
# ==========================================================
returns = 100 * np.log(close_prices / close_prices.shift(1))  # 对数收益率 ×100 变百分比
returns = returns.dropna()

print("\n收益率样本（前5个）：")
print(returns.head())

# ==========================================================
#                     收益率基本统计信息
# ==========================================================
print("\n收益率基本信息：")
print(returns.describe())

skewness = stats.skew(returns)
kurtosis_excess = stats.kurtosis(returns)
kurtosis = stats.kurtosis(returns, fisher=False)

print("偏度：", skewness)
print("超额峰度：", kurtosis_excess)
print("通常峰度：", kurtosis)

# -----------------------------
# 1. User configuration / parameters
# -----------------------------
THETA_LEFT = -1.4395        # 你的阈值（示例）
THETA_RIGHT = 1.171         # 你的阈值（示例）

# 模型参数（示例：替换为你的MLE估计值）
params = {
    "xi": 0.14786087346352017,
    "omega": 1.761869306368757,
    "alpha": -0.016032097147510645,
    "xi_t": -0.014085253434022872,
    "omega_t": 2.6869662613419854,
    "alpha_t": -0.03338510659470208,
    "nu": 3.2529685162075386,
    "theta1": THETA_LEFT,
    "theta2": THETA_RIGHT
}

# bootstrap 参数（可调）
BOOTSTRAP_N = 500    # 可改为 200 或 1000 根据需要
RNG_SEED = 12345

# -----------------------------
# 2. kernel PDFs (vectorized)
# -----------------------------
def skew_normal_pdf_vals(xs, xi, omega, alpha):
    z = (xs - xi) / omega
    return (2.0 / omega) * norm.pdf(z) * norm.cdf(alpha * z)

def skew_t_pdf_vals(xs, xi_t, omega_t, alpha_t, nu):
    z = (xs - xi_t) / omega_t
    t_pdf_z = t.pdf(z, df=nu)
    a_arr = alpha_t * z * np.sqrt((nu + 1) / (nu + z**2))
    t_cdf_a = t.cdf(a_arr, df=nu + 1)
    return (2.0 / omega_t) * t_pdf_z * t_cdf_a

# -----------------------------
# 3. Fast vectorized composite CDF grid builder
# -----------------------------
def build_composite_cdf_grid_fast(x_min, x_max, xi, omega, alpha,
                                 xi_t, omega_t, alpha_t, nu,
                                 theta1, theta2, n_grid=1200):
    """
    Fast vectorized construction of composite CDF using trapezoidal integration on a grid.
    Returns: interp_func, xs, cdf_vals, comp_pdf
    """
    xs = np.linspace(x_min, x_max, n_grid)
    dx = xs[1] - xs[0]

    # vectorized kernel pdfs on grid
    skewt_vals = skew_t_pdf_vals(xs, xi_t, omega_t, alpha_t, nu)
    skewnorm_vals = skew_normal_pdf_vals(xs, xi, omega, alpha)

    # cumulative trapezoid for kernel CDFs (approx)
    # We use cumulative trapezoid manually: cdf[i] = sum_{j< i} 0.5*(f_j + f_{j+1})*dx
    cdf_skewt = np.empty_like(xs)
    cdf_skewt[0] = 0.0
    cdf_skewt[1:] = np.cumsum(0.5 * (skewt_vals[:-1] + skewt_vals[1:]) * dx)

    cdf_skewn = np.empty_like(xs)
    cdf_skewn[0] = 0.0
    cdf_skewn[1:] = np.cumsum(0.5 * (skewnorm_vals[:-1] + skewnorm_vals[1:]) * dx)

    # get threshold-related quantities via interpolation
    theta1_c = np.clip(theta1, x_min, x_max)
    theta2_c = np.clip(theta2, x_min, x_max)
    S_t_theta1 = float(np.interp(theta1_c, xs, cdf_skewt))
    S_t_theta2 = float(np.interp(theta2_c, xs, cdf_skewt))
    S_n_theta1 = float(np.interp(theta1_c, xs, cdf_skewn))
    S_n_theta2 = float(np.interp(theta2_c, xs, cdf_skewn))

    s_t_theta1 = float(np.interp(theta1_c, xs, skewt_vals))
    s_t_theta2 = float(np.interp(theta2_c, xs, skewt_vals))
    s_n_theta1 = float(np.interp(theta1_c, xs, skewnorm_vals))
    s_n_theta2 = float(np.interp(theta2_c, xs, skewnorm_vals))

    # mixing weights r1, r2 (vector quantities are scalars here)
    numerator_r1 = s_t_theta2 * s_n_theta1 * S_t_theta1
    denominator_r1 = (s_t_theta1 * s_t_theta2 * (S_n_theta2 - S_n_theta1)
                      + s_t_theta2 * s_n_theta1 * S_t_theta1
                      + s_t_theta1 * s_n_theta2 * (1.0 - S_t_theta2))
    r1 = numerator_r1 / denominator_r1 if denominator_r1 != 0 else 0.0

    numerator_r2 = s_t_theta1 * (S_n_theta2 - S_n_theta1) + S_t_theta1 * s_n_theta1
    denominator_r2 = S_t_theta1 * s_n_theta1 if (S_t_theta1 * s_n_theta1) != 0 else 1.0
    r2 = 1.0 - r1 * (numerator_r2 / denominator_r2)

    # component masks on grid
    left_mask = xs <= theta1
    center_mask = (xs > theta1) & (xs < theta2)
    right_mask = xs >= theta2

    denom_left = S_t_theta1 if S_t_theta1 > 0 else 1e-12
    denom_center = (S_n_theta2 - S_n_theta1) if (S_n_theta2 - S_n_theta1) > 0 else 1e-12
    denom_right = (1.0 - S_t_theta2) if (1.0 - S_t_theta2) > 0 else 1e-12

    comp_pdf = np.empty_like(xs)
    comp_pdf[left_mask] = (r1 / denom_left) * skewt_vals[left_mask]
    comp_pdf[center_mask] = ((1.0 - r1 - r2) / denom_center) * skewnorm_vals[center_mask]
    comp_pdf[right_mask] = (r2 / denom_right) * skewt_vals[right_mask]

    # cumulative trapezoid for composite cdf
    cdf_vals = np.empty_like(xs)
    cdf_vals[0] = 0.0
    cdf_vals[1:] = np.cumsum(0.5 * (comp_pdf[:-1] + comp_pdf[1:]) * dx)
    # normalize numerically
    cdf_vals = np.clip(cdf_vals / cdf_vals[-1], 0.0, 1.0)

    interp_func = interp1d(xs, cdf_vals, kind='linear', bounds_error=False, fill_value=(0.0,1.0))
    return interp_func, xs, cdf_vals, comp_pdf

# -----------------------------
# 4. Tail-KS using conditional tail CDFs
# -----------------------------
def tail_ks_with_model(sample, model_cdf_interp, tail='right', theta=None, return_path=False):
    sample = np.asarray(sample)
    if tail == 'left':
        mask = sample <= theta
        sample_tail = np.sort(sample[mask])
        if sample_tail.size == 0:
            raise ValueError("No observations in left tail <= theta")
        n = len(sample_tail)
        ecdf = np.arange(1, n+1) / n
        F_theta = float(model_cdf_interp(theta))
        if F_theta <= 0:
            raise ValueError("Model F(theta) <= 0, cannot normalize left tail")
        model_vals = np.array([float(model_cdf_interp(x)) / F_theta for x in sample_tail])
        D_vals = np.abs(ecdf - model_vals)
        idx = int(np.argmax(D_vals))
        D = float(D_vals[idx])
        if return_path:
            return D, sample_tail[idx], idx, sample_tail, ecdf, model_vals
        else:
            return D, sample_tail[idx], idx

    elif tail == 'right':
        mask = sample >= theta
        sample_tail = np.sort(sample[mask])
        if sample_tail.size == 0:
            raise ValueError("No observations in right tail >= theta")
        n = len(sample_tail)
        ecdf = np.arange(1, n+1) / n
        F_theta = float(model_cdf_interp(theta))
        denom = 1.0 - F_theta
        if denom <= 0:
            raise ValueError("Model F(theta) >= 1, cannot normalize right tail")
        model_vals = np.array([(float(model_cdf_interp(x)) - F_theta) / denom for x in sample_tail])
        D_vals = np.abs(ecdf - model_vals)
        idx = int(np.argmax(D_vals))
        D = float(D_vals[idx])
        if return_path:
            return D, sample_tail[idx], idx, sample_tail, ecdf, model_vals
        else:
            return D, sample_tail[idx], idx
    else:
        raise ValueError("tail must be 'left' or 'right'")

# -----------------------------
# 5. Fast inversion sampling from grid CDF
# -----------------------------
def sample_from_model_grid(n, grid_xs, grid_cdf):
    eps = 1e-12
    gc = np.clip(grid_cdf, eps, 1-eps)
    inv_interp = interp1d(gc, grid_xs, bounds_error=False, fill_value=(grid_xs[0], grid_xs[-1]))
    u = np.random.rand(n)
    return inv_interp(u)

# -----------------------------
# 6. Parallel bootstrap for Tail-KS p-value
# -----------------------------
def _bootstrap_one(args):
    n, grid_xs, grid_cdf, tail, theta, model_cdf_interp = args
    sim = sample_from_model_grid(n, grid_xs, grid_cdf)
    if tail == 'left':
        sim_tail = np.sort(sim[sim <= theta])
        if len(sim_tail) < n:
            extra = sample_from_model_grid(n, grid_xs, grid_cdf)
            sim = np.concatenate([sim, extra])
            sim_tail = np.sort(sim[sim <= theta])[:n]
        else:
            sim_tail = sim_tail[:n]
        D_b, _, _ = tail_ks_with_model(sim_tail, model_cdf_interp, tail='left', theta=theta)
    else:
        sim_tail = np.sort(sim[sim >= theta])
        if len(sim_tail) < n:
            extra = sample_from_model_grid(n, grid_xs, grid_cdf)
            sim = np.concatenate([sim, extra])
            sim_tail = np.sort(sim[sim >= theta])[-n:]
        else:
            sim_tail = sim_tail[-n:]
        D_b, _, _ = tail_ks_with_model(sim_tail, model_cdf_interp, tail='right', theta=theta)
    return D_b

def tail_ks_bootstrap_pvalue_parallel(sample, model_cdf_interp, grid_xs, grid_cdf, tail, theta,
                                      B=500, rng_seed=None, n_jobs=None):
    np.random.seed(rng_seed)
    D_obs, _, _ = tail_ks_with_model(sample, model_cdf_interp, tail=tail, theta=theta)
    n = len(sample[(sample <= theta) if tail == 'left' else (sample >= theta)])
    if n <= 0:
        raise ValueError("No tail observations to bootstrap.")
    n_jobs = n_jobs or max(1, cpu_count() - 1)
    args_list = [(n, grid_xs, grid_cdf, tail, theta, model_cdf_interp) for _ in range(B)]
    with Pool(processes=n_jobs) as pool:
        D_boot_list = pool.map(_bootstrap_one, args_list)
    D_boot = np.array(D_boot_list)
    p_value = np.mean(D_boot >= D_obs)
    return D_obs, D_boot, p_value

# -----------------------------
# 7. Main workflow
# -----------------------------
def main():
    # use returns computed at top (pandas Series)
    returns_arr = returns.values
    print("Number of returns:", len(returns_arr))
    print("Basic stats (returns): mean {:.4f}, std {:.4f}".format(np.mean(returns_arr), np.std(returns_arr)))

    # build grid bounds
    x_min = np.percentile(returns_arr, 0.1) - 1.0
    x_max = np.percentile(returns_arr, 99.9) + 1.0
    print("Building composite CDF grid on [{:.3f}, {:.3f}] ...".format(x_min, x_max))

    interp_func, xs_grid, ys_grid, comp_pdf = build_composite_cdf_grid_fast(
        x_min, x_max,
        params["xi"], params["omega"], params["alpha"],
        params["xi_t"], params["omega_t"], params["alpha_t"], params["nu"],
        params["theta1"], params["theta2"],
        n_grid=1200
    )
    print("Grid built: {} points.".format(len(xs_grid)))

    # compute Tail-KS for left and right tails
    D_left, xmax_left, idx_left, sample_tail_left, ecdf_left, model_left = tail_ks_with_model(
        returns_arr, interp_func, tail='left', theta=params["theta1"], return_path=True)
    print("Left-tail KS D = {:.6f} at x = {:.4f} ({} tail obs)".format(D_left, xmax_left, len(sample_tail_left)))

    D_right, x_at_max_right, idx_right, sample_tail_right, ecdf_right, model_right = tail_ks_with_model(
        returns_arr, interp_func, tail='right', theta=params["theta2"], return_path=True)
    print("Right-tail KS D = {:.6f} at x = {:.4f} ({} tail obs)".format(D_right, x_at_max_right, len(sample_tail_right)))

    # plot ECDF vs model conditional CDF for tails
    fig, axes = plt.subplots(1,2, figsize=(12,5))
    axes[0].step(sample_tail_left, ecdf_left, where='post', label='Empirical ECDF (left tail)')
    axes[0].plot(sample_tail_left, model_left, 'r.-', label='Model conditional CDF (left tail)')
    axes[0].set_title("Left tail: D = {:.4f}".format(D_left))
    axes[0].legend()
    axes[0].grid(True)

    axes[1].step(sample_tail_right, ecdf_right, where='post', label='Empirical ECDF (right tail)')
    axes[1].plot(sample_tail_right, model_right, 'r.-', label='Model conditional CDF (right tail)')
    axes[1].set_title("Right tail: D = {:.4f}".format(D_right))
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

    # bootstrap p-values (parallel)
    print("Running parallel bootstrap (B={}) for left tail ...".format(BOOTSTRAP_N))
    Dleft_obs, Dleft_boot, pleft = tail_ks_bootstrap_pvalue_parallel(
        returns_arr, interp_func, xs_grid, ys_grid, tail='left', theta=params["theta1"],
        B=BOOTSTRAP_N, rng_seed=RNG_SEED, n_jobs=None)
    print("Left tail bootstrap p-value (Tail-KS) = {:.4f}".format(pleft))

    print("Running parallel bootstrap (B={}) for right tail ...".format(BOOTSTRAP_N))
    Dright_obs, Dright_boot, pright = tail_ks_bootstrap_pvalue_parallel(
        returns_arr, interp_func, xs_grid, ys_grid, tail='right', theta=params["theta2"],
        B=BOOTSTRAP_N, rng_seed=RNG_SEED+1, n_jobs=None)
    print("Right tail bootstrap p-value (Tail-KS) = {:.4f}".format(pright))

    # optionally save results
    # np.savez("tailks_results_fast.npz", D_left=D_left, D_right=D_right, Dleft_boot=Dleft_boot, Dright_boot=Dright_boot)

if __name__ == "__main__":
    main()